## import / load data

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from tqdm import notebook
import gc

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import lightgbm
import xgboost


C:\Users\User\Anaconda3\lib\site-packages\distributed\config.py:63: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config.update(yaml.load(text) or {})


In [2]:
path = './data/'
train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')
submission  = pd.read_csv(path + 'submission.csv')

## Data setting

In [3]:
# train/test 결합
train['test'] = 0
test['test'] = 1
merge_df = pd.concat([train,test])

In [4]:
# 날짜
merge_df['date'] = pd.to_datetime(merge_df.date)
merge_df['year'] = merge_df['date'].apply(lambda x : x.year)
merge_df['month'] = merge_df['date'].apply(lambda x : x.month)

In [5]:
# 평균소비액
daily_amount = merge_df[merge_df.amount>0].groupby('date')['amount'].mean()

# 거래취소의 경우 
merge_df.loc[merge_df.amount <0, 'refund'] = 1
merge_df.loc[merge_df.amount >0, 'refund'] = 0
# merge_df.loc[merge_df.amount <0, 'amount'] = 0

In [6]:
merge_df.head(2)

,store_id,date,time,card_id,amount,installments,days_of_week,holyday,test,year,month,refund
0,0,2016-12-14,18:05:31,d297bba73f,5,NaN,2,0,0,2016,12,0.0
1,0,2016-12-14,18:05:54,d297bba73f,-5,NaN,2,0,0,2016,12,1.0


### train_data setting
* store_id가 연속적이진 않다 (11과 같은 숫자는 없음)
* 거래금액이 0인 경우도 현재 포함되어 있음

In [263]:
# 전체 날짜의 요일, 휴일
all_dates = merge_df.drop_duplicates('date').sort_values('date')
all_dates = all_dates[['date','days_of_week','holyday']]
all_dates.reset_index(drop=True, inplace=True)

In [264]:
train_df = merge_df[merge_df.test == 0]

# 날짜범위 확인
train_fin = train_df.groupby('store_id')['date'].agg(['min','max'])
train_fin.reset_index(inplace=True)
train_fin['last_day'] = train_fin['max'] - datetime.timedelta(100)

#최초거래일자부터 마지막 거래일자까지의 기간
train_fin['day_gap'] = train_fin['last_day'] - train_fin['min']
train_fin['day_gap'] = train_fin['day_gap'].apply(lambda x : x.days)

# 마지막 날짜에서 100일전 이전의 거래내역이 없으면 학습대상이 아님
train_fin.loc[train_fin['min'] > train_fin['last_day'],'outbound'] = 1
train_fin.loc[~(train_fin['min'] > train_fin['last_day']),'outbound'] = 0

del_list = train_fin[train_fin.outbound == 1].index

train_fin.drop(del_list,inplace= True)
del train_fin['outbound']
train_fin.reset_index(drop = True,inplace=True)

check_amount = train_df.groupby(['store_id','date'])['amount'].agg(['sum','count']).reset_index()
check_amount = pd.merge(check_amount,all_dates)
refund_df = train_df.groupby(['store_id','date'])['refund'].sum().reset_index()
installlment_df = train_df.groupby(['store_id','date'])['installments'].sum().reset_index()

In [265]:
before_amount_ls = []
after_amount_ls = []
transfer_count_ls = []

before_holyday_ls = []
before_week_dict_ls =[]
refund_ls = []
installment_ls = []
installment_count_ls =[]
holy_amount_ls = []
week_amount_ls = []

for i in notebook.tqdm(train_fin.index):
    last_day = train_fin.loc[train_fin.index == i,'last_day'].iloc[0]
    store_id = train_fin.loc[train_fin.index == i,'store_id'].iloc[0]
    one = check_amount.loc[(check_amount.store_id == store_id)].reset_index(drop=True)
    two = refund_df.loc[(refund_df.store_id == store_id)].reset_index(drop=True)
    three = installlment_df.loc[(installlment_df.store_id == store_id)].reset_index(drop=True)
    
    last_index = one.loc[one.date < last_day].index[-1]

    before = one[0:last_index+1]
    after = one[last_index+1:]

    transfer_count = before['count'].sum()
    before_amount = before['sum'].sum()
    after_amount = after['sum'].sum()


    #요일, 휴일
    before_holyday = pd.merge(before,all_dates)['holyday'].sum()
    before_week = pd.merge(before,all_dates)['days_of_week'].value_counts().to_dict()

    after_holyday = pd.merge(after,all_dates)['holyday'].sum()
    after_week = pd.merge(after,all_dates)['days_of_week'].value_counts().to_dict()

    #환불횟수, 할부총기간
    refund = two[0:last_index+1]['refund'].sum()
    installment = three[0:last_index+1]['installments'].sum()
    installment_count = (three[0:last_index+1]['installments']>0).sum()

    #휴일여부, 요일별 거래금액
    holy_amount = one.groupby('holyday')['sum'].sum().to_dict()
    week_amount = one.groupby('days_of_week')['sum'].sum().to_dict()

    #저장
    transfer_count_ls.append(transfer_count)
    before_amount_ls.append(before_amount)
    after_amount_ls.append(after_amount)

    before_holyday_ls.append(before_holyday)
    before_week_dict_ls.append(before_week)

    refund_ls.append(refund)
    installment_ls.append(installment)

    holy_amount_ls.append(holy_amount)
    week_amount_ls.append(week_amount)
    installment_count_ls.append(installment_count)
        
train_fin['transfer_count'] = transfer_count_ls
train_fin['before_amount'] = before_amount_ls
train_fin['after_amount'] = after_amount_ls
train_fin['holyday'] = before_holyday_ls
train_fin['refund_sum'] = refund_ls
train_fin['installment_sum'] = installment_ls
train_fin['installment_count'] = installment_count_ls

# 요일 추가
week_df = pd.DataFrame(before_week_dict_ls)
week_df = week_df[[0,1,2,3,4,5,6]]
train_fin = pd.concat([train_fin,week_df],axis= 1)

# 공휴일, 요일별 매출액
holy_amount_df = pd.DataFrame(holy_amount_ls)
holy_amount_df.rename(columns = {0:'unholy_amount',1:'holy_amount'},inplace = True)
train_fin = pd.concat([train_fin,holy_amount_df],axis= 1)

week_amount_df = pd.DataFrame(week_amount_ls)
week_amount_df.rename(columns = {0:"0_amount",1:"1_amount",2:"2_amount",3:"3_amount",4:"4_amount",5:"5_amount",6:"6_amount"},inplace = True)
train_fin = pd.concat([train_fin,week_amount_df],axis= 1)

In [266]:
#이후 100일까지의 공휴일과 요일 

after_holy_ls = []
after_week_ls = []
for i in notebook.tqdm(range(0, len(train_fin))):
    start_day = train_fin.loc[i, 'last_day']
    end_day = train_fin.loc[i, 'max']
    target = all_dates[(all_dates.date > start_day)&(all_dates.date <= end_day)]
    after_holy = target.holyday.sum()
    after_week = target.days_of_week.value_counts().to_dict()
    
    after_holy_ls.append(after_holy)
    after_week_ls.append(after_week)

week_df = pd.DataFrame(after_week_ls)
week_df = week_df[[0,1,2,3,4,5,6]]
week_df.rename(columns={0:"a_0",1:"a_1",2:"a_2",3:"a_3",4:"a_4",5:"a_5",6:"a_6"},inplace=True)

train_fin['after_holy'] = after_holy_ls
train_fin = pd.concat([train_fin,week_df],axis = 1)

In [267]:
train_fin.head(2)

,store_id,min,max,last_day,day_gap,transfer_count,before_amount,after_amount,holyday,refund_sum,...,5_amount,6_amount,after_holy,a_0,a_1,a_2,a_3,a_4,a_5,a_6
0,0,2016-12-14,2018-07-31,2018-04-22,494,56725,4361581,873030,25,499.0,...,802135.0,837312.0,6,15,15,14,14,14,14,14
1,1,2016-12-21,2018-07-02,2018-03-24,458,35,120650,27300,0,0.0,...,12000.0,NaN,6,15,14,14,14,14,14,15


### test_data setting

In [268]:
test_df = merge_df[merge_df.test == 1]

# 날짜범위 확인
test_fin = test_df.groupby('store_id')['date'].agg(['min','max'])
test_fin.reset_index(inplace=True)

#최초거래일자부터 마지막 거래일자까지의 기간
test_fin['day_gap'] = test_fin['max'] - test_fin['min']
test_fin['day_gap'] = test_fin['day_gap'].apply(lambda x : x.days)

# 일자별 결제
check_amount = test_df.groupby(['store_id'])['amount'].agg(['sum','count']).reset_index()
check_amount = check_amount.rename(columns={'sum':'before_amount','count':'transfer_count'})
test_fin = pd.merge(test_fin,check_amount)

refund_df = test_df.groupby(['store_id','date'])['refund'].sum().reset_index()
installlment_df = test_df.groupby(['store_id','date'])['installments'].sum().reset_index()

In [269]:
check_amount = train_df.groupby(['store_id','date'])['amount'].sum().reset_index()

refund_ls = []
installment_ls = []
installment_count_ls =[]

before_holyday_ls = []
before_week_dict_ls =[]

holy_amount_ls = []
week_amount_ls = []

for store_id in notebook.tqdm(range(0, test_fin.store_id.nunique())):
    one = check_amount.loc[(check_amount.store_id == store_id)].reset_index(drop=True)
    two = refund_df.loc[(refund_df.store_id == store_id)].reset_index(drop=True)
    three = installlment_df.loc[(installlment_df.store_id == store_id)].reset_index(drop=True)
    
    refund = two['refund'].sum()
    installment = three['installments'].sum()
    installment_count = (three['installments']>0).sum()
    
    holyday = pd.merge(one['date'],all_dates)['holyday'].sum()
    before_week = pd.merge(one['date'],all_dates)['days_of_week'].value_counts().to_dict()
    
    #휴일 여부에 따른 매출액
    holy_amount = test_df[test_df.store_id == store_id].groupby('holyday')['amount'].sum().to_dict()
    week_amount = test_df[test_df.store_id == store_id].groupby('days_of_week')['amount'].sum().to_dict()
    
    refund_ls.append(refund)
    installment_ls.append(installment)
    
    before_holyday_ls.append(holyday)
    before_week_dict_ls.append(before_week)
    
    holy_amount_ls.append(holy_amount)
    week_amount_ls.append(week_amount)
    installment_count_ls.append(installment_count)
    
test_fin['holyday'] =  before_holyday_ls
test_fin['refund_sum'] =  refund_ls
test_fin['installment_sum'] = installment_ls
test_fin['installment_count'] = installment_count_ls

# 요일 추가
week_df = pd.DataFrame(before_week_dict_ls)
week_df = week_df[[0,1,2,3,4,5,6]]

test_fin = pd.concat([test_fin,week_df],axis= 1)

# 공휴일, 요일별 매출액
holy_amount_df = pd.DataFrame(holy_amount_ls)
holy_amount_df.rename(columns = {0:'unholy_amount',1:'holy_amount'},inplace = True)
test_fin = pd.concat([test_fin,holy_amount_df],axis= 1)

week_amount_df = pd.DataFrame(week_amount_ls)
week_amount_df.rename(columns = {0:"0_amount",1:"1_amount",2:"2_amount",3:"3_amount",4:"4_amount",5:"5_amount",6:"6_amount"},inplace = True)
test_fin = pd.concat([test_fin,week_amount_df],axis= 1)

In [270]:
#이후 100일까지의 공휴일과 요일 
test_fin['after_100'] = test_fin['max']+datetime.timedelta(100)

after_holy_ls = []
after_week_ls = []
for i in notebook.tqdm(range(0, len(test_fin))):
    start_day = test_fin.loc[i, 'max']
    end_day = test_fin.loc[i, 'after_100']
    target = all_dates[(all_dates.date > start_day)&(all_dates.date <= end_day)]
    after_holy = target.holyday.sum()
    after_week = target.days_of_week.value_counts().to_dict()
    
    after_holy_ls.append(after_holy)
    after_week_ls.append(after_week)

week_df = pd.DataFrame(after_week_ls)
week_df = week_df[[0,1,2,3,4,5,6]]
week_df.rename(columns={0:"a_0",1:"a_1",2:"a_2",3:"a_3",4:"a_4",5:"a_5",6:"a_6"},inplace=True)

test_fin['after_holy'] = after_holy_ls
test_fin = pd.concat([test_fin,week_df],axis = 1)

In [271]:
test_fin.head(2)

,store_id,min,max,day_gap,before_amount,transfer_count,holyday,refund_sum,installment_sum,installment_count,...,6_amount,after_100,after_holy,a_0,a_1,a_2,a_3,a_4,a_5,a_6
0,0,2016-08-01,2018-03-31,607,638257,4215,31,14.0,2.0,1,...,99337.0,2018-07-09,6,15,14,14,14,14,14,15
1,1,2016-08-02,2018-03-30,605,427806,435,0,14.0,192.0,49,...,51855.0,2018-07-08,6,14,14,14,14,14,15,15


### 추가변수

In [272]:
def add_feature(df):
    '''추가변수 생성 
    거래횟수당 평균 금액, 일평균 금액, 거래횟수 대비 환불비율
    휴일 평균 거래금액, 평일 평균 거래금액
    요일별 평균 거래금액
    
    '''
    df['refund_per_all'] = df['refund_sum'] / df['transfer_count']
    df['installment_mean'] = df['installment_sum'] / df['installment_count']
    
    df['trans_per_amount'] = df['before_amount'] / df['transfer_count']
    df['day_per_amount'] = df['before_amount'] / df['day_gap']
    
    df['holy_amount_mean'] = df['holy_amount'] / df['holyday']
    df['unholy_amount_mean'] = df['unholy_amount'] / (df['day_gap'] - df['holyday'])
    
    for i in range(0, 7):
        df[f'{i}_amount_mean'] = df[f'{i}_amount'] / df[i]
    
    #후처리
    df = df.applymap(lambda x : 0 if x == np.inf else x)
    df.fillna(0,inplace=True)
    return df

In [273]:
test_fin = add_feature(test_fin)
train_fin = add_feature(train_fin)

### 최종전처리

In [274]:
y = train_fin['after_amount']

train_df = train_fin.drop(['min','max','last_day','after_amount','store_id'],axis = 1)
test_df = test_fin.drop(['min','max','after_100','store_id'],axis = 1)

In [275]:
train_df.shape, test_df.shape

((1641, 44), (200, 44))

In [276]:
train_df.columns

Index([           'day_gap',     'transfer_count',      'before_amount',
                  'holyday',         'refund_sum',    'installment_sum',
        'installment_count',                    0,                    1,
                          2,                    3,                    4,
                          5,                    6,      'unholy_amount',
              'holy_amount',           '0_amount',           '1_amount',
                 '2_amount',           '3_amount',           '4_amount',
                 '5_amount',           '6_amount',         'after_holy',
                      'a_0',                'a_1',                'a_2',
                      'a_3',                'a_4',                'a_5',
                      'a_6',     'refund_per_all',   'installment_mean',
         'trans_per_amount',     'day_per_amount',   'holy_amount_mean',
       'unholy_amount_mean',      '0_amount_mean',      '1_amount_mean',
            '2_amount_mean',      '3_amount_mean', 

In [277]:
to_log_ls = ['before_amount', 'trans_per_amount',     'day_per_amount',
             'holy_amount_mean', 'unholy_amount_mean',      '0_amount_mean',
             '1_amount_mean',      '2_amount_mean',      '3_amount_mean',
             '4_amount_mean',      '5_amount_mean',      '6_amount_mean']

In [278]:
#로그 변환
for col in to_log_ls:
    train_df[col] = np.log(1+train_df[col])
    test_df[col] = np.log(1+test_df[col])

C:\Users\User\AppData\Roaming\Python\Python36\site-packages\pandas\core\series.py:853: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [279]:
train_df.fillna(0, inplace=True)
test_df.fillna(0, inplace=True)

### train_test split

In [280]:
X_train, X_test, y_train, y_test = train_test_split(train_df,y,test_size =0.3, random_state = 42)

### modeling 

#### rf

In [281]:
rf = RandomForestRegressor(random_state=42, n_estimators= 1000)
rf.fit(X_train,y_train)
rf_y_pred = rf.predict(X_test)

np.sqrt(mean_squared_error(y_test,rf_y_pred))

14611.044528954015

In [282]:
fi_df = pd.DataFrame(rf.feature_importances_*100,index = train_df.columns).sort_values(0, ascending = False)
fi_df

,0
holy_amount_mean,18.420461
day_per_amount,16.421179
unholy_amount_mean,7.846433
3_amount_mean,7.639428
4_amount_mean,7.418639
0_amount_mean,7.145091
2_amount_mean,6.995511
day_gap,3.989246
5_amount_mean,3.510745
6_amount_mean,2.812684


#### lgbm

In [283]:
lgbm_c = lightgbm.LGBMRegressor(random_state=42, learning_rate=0.005,n_estimators= 1000,)
lgbm_c.fit(X_train,y_train)
lgbm_y_pred = lgbm_c.predict(X_test)

np.sqrt(mean_squared_error(y_test,lgbm_y_pred))

29393.138022876672

In [284]:
fi_df = pd.DataFrame(lgbm_c.feature_importances_,index = train_df.columns).sort_values(0, ascending = False)
fi_df

,0
unholy_amount_mean,3001
day_gap,2936
day_per_amount,1811
5_amount_mean,1748
unholy_amount,1497
2_amount_mean,1418
holy_amount_mean,1341
before_amount,1310
trans_per_amount,1309
1_amount_mean,1253


### final

In [304]:
# #lgbm
# lgbm_c.fit(train_df,y)
# y_pred_fin = lgbm_c.predict(test_df)
# score = np.sqrt(mean_squared_error(y_test,lgbm_y_pred))

# rf
rf.fit(train_df,y)
y_pred_fin = rf.predict(test_df)
score = np.sqrt(mean_squared_error(y_test,rf_y_pred))

In [305]:
submission['total_sales'] = y_pred_fin

today = datetime.datetime.today().date()
os.makedirs('./submission/',exist_ok=True)
submission.to_csv(f'./submission/submission_{today}_rf_{score}.csv',index=False)

In [298]:
today = datetime.datetime.today()

In [301]:
datetime.datetime.date(today)

datetime.date(2020, 6, 22)